In [1]:
import init
import epsilon_rho
import metrics
import optimize_algorithm as optimize
import numpy as np

In [2]:
def normalize_density_matrix(rho):
    # Calculate the trace of the density matrix
    trace = np.trace(rho)
    
    # Normalize the density matrix
    if trace != 0:
        normalized_rho = rho / trace
    else:
        raise ValueError("The density matrix has a trace of zero and cannot be normalized.")
    
    return normalized_rho

In [5]:
def quantum_process_tomography_dephasing (num_qubits, gamma, alpha, num_loop, rho, unitary, mode='fidelity'):
    rho2 = epsilon_rho.calculate_dephasing(rho, num_qubits, gamma)
    # Optimizing
    out_unitary, costdict = optimize.optimize_adam_unitary_dagger(rho=rho, rho2=rho2, unitary=unitary, num_qubits=num_qubits, alpha=alpha, num_loop=num_loop, mode=mode)
    
    _cost = costdict[-1]
    return rho2, out_unitary, _cost, costdict

#Plot lost theo iteration với n khác nhau
#Plot 1-fidelity(rho, rho3) và 1-fidelity(rho2, rho2) theo các n khác nhau (plot điểm)
#Plot rho2 và rho2_kraus (cho n = 1)
def quantum_process_tomography_unitary (num_qubits, num_loop, alpha, rho, kraus_operators, unitary, mode='fidelity'):
    
    # Optimizing
    out_kraus_operators, costdict = optimize.optimize_adam_kraus(rho=rho, kraus_operators=kraus_operators, unitary=unitary, num_qubits=num_qubits, alpha=alpha, num_loop=num_loop, mode=mode)
    print(mode)
    rho2 = epsilon_rho.calculate_from_kraus_operators(rho, out_kraus_operators)
    rho3 = epsilon_rho.calculate_from_unitary_dagger(rho2, unitary)
    _cost = costdict[-1]
    return rho2, out_kraus_operators, _cost, costdict, rho3


In [6]:
import os
from qiskit.quantum_info import DensityMatrix
np.set_printoptions(threshold=np.inf)
import re
def write_result (folder, num_qubits, g, alpha, num_loop, rho, unitary):
        folder_path = str(num_qubits)+"_qubits_"+"{:.3f}".format(g)
        folder_path = os.path.join(folder, folder_path)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        rho2, out_unitary, _cost, costdict = quantum_process_tomography_dephasing(num_qubits, g, alpha, num_loop, rho, unitary)
        
        rho2_file_path = os.path.join(folder_path, 'rho2.txt')
        unitary_file_path = os.path.join(folder_path, 'unitary.txt')
        cost_file_path = os.path.join(folder_path, 'cost.txt')
        fidelity22_file_path = os.path.join(folder_path, 'fidelity_rho2_rho2.txt')
        fidelity03_file_path = os.path.join(folder_path, 'fidelity_rho_rho3.txt')
        costdict_file_path = os.path.join(folder_path, 'costdict.txt')

        print(g, num_qubits, _cost)
        
        rho3 = epsilon_rho.calculate_from_unitary_dagger(rho2, out_unitary)
        out_rho2 = epsilon_rho.calculate_from_unitary(rho, out_unitary)

        out_rho2 = normalize_density_matrix(out_rho2)
        #rho3 = normalize_density_matrix(rho3)
        
        with open(rho2_file_path, 'w') as file:
            file.write(str(rho2))
        with open(unitary_file_path, 'w') as file:
            file.write(str(out_unitary.numpy()))
        with open(cost_file_path, 'w') as file:
            file.write(str(_cost.real))
        with open(fidelity03_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho, rho3).numpy()))
        with open(fidelity22_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho2, out_rho2).numpy()))
        with open(costdict_file_path, 'w') as file:
            file.write(str(costdict))


def write_result_unitary (folder, num_qubits, alpha):
        folder_path = str(num_qubits)+"_qubits"
        folder_path = os.path.join(folder, folder_path)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        rho = init.create_random_state(num_qubits=num_qubits)
        unitary_matrix = init.create_unitary_matrix(num_qubits=num_qubits)
        kraus_operators = init.create_kraus_operators_from_unitary(unitary_matrix=unitary_matrix)
    
        unitary = init.create_unitary_matrix(num_qubits=num_qubits)
        

        rho2, out_kraus_operators, _cost, costdict, rho3 = quantum_process_tomography_unitary(num_qubits, 1000, alpha, rho.data, kraus_operators, unitary)
        
        rho_file_path = os.path.join(folder_path, "rho.txt")
        rho2_file_path = os.path.join(folder_path, 'rho2.txt')
        out_rho2_file_path = os.path.join(folder_path, 'out_rho2.txt')
        rho3_file_path = os.path.join(folder_path, 'rho3.txt')
        kraus_file_path = os.path.join(folder_path, 'kraus_operators.txt')
        cost_file_path = os.path.join(folder_path, 'cost.txt')
        fidelity22_file_path = os.path.join(folder_path, 'fidelity_rho2_rho2.txt')
        fidelity03_file_path = os.path.join(folder_path, 'fidelity_rho_rho3.txt')
        costdict_file_path = os.path.join(folder_path, 'costdict.txt')
        unitary_file_path = os.path.join(folder_path, "unitary.txt")

        print(num_qubits, _cost)
        out_rho2 = epsilon_rho.calculate_from_kraus_operators(rho, out_kraus_operators)
        out_rho2 = normalize_density_matrix(out_rho2)
        #rho3 = normalize_density_matrix(rho3)
        
        with open(fidelity03_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho, rho3).numpy()))
        with open(fidelity22_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho2, out_rho2).numpy()))        
        with open(unitary_file_path, 'w') as file:
            file.write(str(unitary))
        with open(rho_file_path, 'w') as file:
            file.write(str(rho.data))
        with open(rho2_file_path, 'w') as file:
            file.write(str(rho2))
        with open(out_rho2_file_path, 'w') as file:
            file.write(str(out_rho2.numpy()))
        with open(rho3_file_path, 'w') as file:
            file.write(str(rho3.numpy()))
        with open(kraus_file_path, 'w') as file:
            file.write(str(out_kraus_operators.numpy()))
        with open(cost_file_path, 'w') as file:
            file.write(str(_cost))
        
        with open(costdict_file_path, 'w') as file:
            file.write(str(costdict))





In [ ]:
experiment_folder = ''
g_shift = 0.05

for num_qubits in range(1, 2):
    if (experiment_folder == ''):
        break
    else:
        if not os.path.exists(experiment_folder):
            os.makedirs(experiment_folder)

    write_result_unitary(experiment_folder, num_qubits, 0.03)
    
    

0.27709088066591825
0.20156208140527965
0.12122399222390878
0.04228340953648757
-0.022485934133067653
-0.06646190236532035
-0.09180798816786773
-0.10077891063035493
-0.09527509881758145
-0.07796821377716778
-0.052534664773260964
-0.023257018743745528
0.005591985800830979
0.030426068938627204
0.04886771296824266
0.059856827643335
0.06338576780095329
0.060139694179196734
0.05123554190120272
0.038084086328690536
0.022322232679995557
0.005752437952427081
-0.009763598888317349
-0.02248670013752907
-0.03104190202128687
-0.03464818794265745
-0.03324239416664865
-0.027448385511404272
-0.018417724188956708
-0.007608190529812209
0.003442096710973619
0.01331258993823603
0.02085503718771098
0.025297528378791845
0.026296123973106322
0.023938682058188387
0.018709276856171297
0.011419996042140612
0.0031119448803250682
-0.005074923692395927
-0.012057330365848662
-0.016969439762732685
-0.019295297978030312
-0.01893867604812227
-0.016212165708188175
-0.011751656945393485
-0.006381755648365184
-0.0009653

In [35]:
experiment_folder = ''
g_shift = 0.05

for num_qubits in range(1, 2):
    if (experiment_folder == ''):
        break
    else:
        if not os.path.exists(experiment_folder):
            os.makedirs(experiment_folder)
    rho = init.create_random_state(num_qubits=num_qubits)
    unitary_matrix = init.create_unitary_matrix(num_qubits=num_qubits)

    rho_file_path = os.path.join(experiment_folder, str(num_qubits)+"_qubits_rho.txt")
    with open(rho_file_path, 'w') as file:
        file.write(str(rho.data))
    g = 1
    while g >= 0:
        write_result(experiment_folder, num_qubits, g, 0.01, 1000, rho, unitary_matrix)
        g = round(g - g_shift, 2)
    
    

1 1 0.11586421409978953
0.95 1 0.10928663513789427
0.9 1 0.10274988102038918
0.85 1 0.09635875018193796
0.8 1 0.09006312485049839
0.75 1 0.08386273117289833
0.7 1 0.07775337608641486
0.65 1 0.07173116351296094
0.6 1 0.0657924675854833
0.55 1 0.05993390711730007
0.5 1 0.054152323091633314
0.45 1 0.048444758745811556
0.4 1 0.042808441893696814
0.35 1 0.03724076918638275
0.3 1 0.03173929205756343
0.25 1 0.0263017041382847
0.2 1 0.020925829957578745
0.15 1 0.015609614771952995
0.1 1 0.010351115388876564
0.05 1 0.005148491868056548
0.0 1 -2.220446049250313e-16


In [7]:
import os
def write_result_2(folder, folder_path, num_qubits, g, alpha, num_loop, rho, unitary): 
    folder_path = os.path.join(folder, folder_path)
    os.makedirs(folder_path, exist_ok=True)  # Create directory if it doesn't exist

    rho2, out_unitary, _cost, costdict = quantum_process_tomography_dephasing(num_qubits, g, alpha, num_loop, rho, unitary)
    
    # File paths
    rho2_file_path = os.path.join(folder_path, 'rho2.txt')
    unitary_file_path = os.path.join(folder_path, 'unitary.txt')
    trace_rho2_file_path = os.path.join(folder_path, 'tr_rho2.txt')
    trace_out_rho2_file_path = os.path.join(folder_path, 'tr_out_rho2.txt')
    fidelity_file_path = os.path.join(folder_path, 'fidelity.txt')

    out_rho2 = epsilon_rho.calculate_from_unitary(rho, out_unitary)
    out_rho2 = normalize_density_matrix(out_rho2)
    
    # Define Pauli-X matrix
    pauli_X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
    # Calculate tr(X1, rho)
    trace_rho2 = metrics.trace_Pauli(rho2, 0, pauli_X)
    trace_out_rho2 = metrics.trace_Pauli(out_rho2, 0, pauli_X)
    
    print(g, num_qubits, _cost)
    print(costdict)
    # Save results
    with open(rho2_file_path, 'w') as file:
        file.write(str(rho2))
    with open(fidelity_file_path, 'w') as file:
        file.write(str(metrics.compilation_trace_fidelity
                       (rho, epsilon_rho.calculate_from_unitary_dagger(rho2, out_unitary)).numpy().real))
    with open(unitary_file_path, 'w') as file:
        file.write(str(out_unitary.numpy()))
    with open(trace_rho2_file_path, 'w') as file:
        file.write(str(trace_rho2.real))
    with open(trace_out_rho2_file_path, 'w') as file:
        file.write(str(trace_out_rho2.real))

In [35]:
from qiskit.quantum_info import DensityMatrix
experiment_folder = ''
# Đổi hàm cost thành trace fidelity
for num_qubits in range(1, 2):
    if (experiment_folder == ''):
        break
    else:
        if not os.path.exists(experiment_folder):
            os.makedirs(experiment_folder)
    rho = DensityMatrix([[0.99817499+0.j, 0.04268112+0.j], [0.04268112+0.j, 0.00182501+0.j]])
    unitary_matrix = [[-0.07737892+0.j,  0.99700176+0.j], [ 0.99700176+0.j,  0.07737892+0.j]]

    rho_file_path = os.path.join(experiment_folder, str(num_qubits)+"_qubits_rho.txt")
    with open(rho_file_path, 'w') as file:
        file.write(str(rho.data))
    unitary_file_path = os.path.join(experiment_folder, str(num_qubits)+"_UNITARY.txt")
    with open(unitary_file_path, 'w') as file:
        file.write(str(unitary_matrix))
    g = 0
    beta = 0.1
    # Create 50 t from 0 to 48
    t_s = np.linspace(74, 100, 53)
    for t in t_s:
        t=round(t, 1)
        print(g, num_qubits, t)
        folder_path_a = str(num_qubits)+"_qubits_"+"{:07.2f}".format(t)+'_a'
        folder_path_b = str(num_qubits)+"_qubits_"+"{:07.2f}".format(t)+'_b'
        write_result_2(experiment_folder, folder_path_a, 
                        num_qubits, 1 - np.exp(-2 * beta * t), 0.01, 700, rho, unitary_matrix)
        write_result_2(experiment_folder, folder_path_b,
                        num_qubits, 1 - np.exp(-2 * beta * t**2), 0.01, 700, rho, unitary_matrix)
    
    


0 1 74.0
0.999999626370062 1 0.0018250106468539196
[0.9961808507057697, 0.9951714577493403, 0.9939470393765268, 0.9925048686630057, 0.990840712740017, 0.9889492074037842, 0.9868241325395044, 0.9844586110089415, 0.9818452541666516, 0.9789762712267674, 0.9758435542480953, 0.9724387465854281, 0.9687533000301031, 0.9647785241415666, 0.9605056301448971, 0.9559257710268023, 0.951030078968728, 0.9458097009238814, 0.9402558329200754, 0.9343597535169865, 0.9281128567417216, 0.9215066847554505, 0.9145329604561993, 0.9071836201919513, 0.899450846739281, 0.8913271026926364, 0.8828051644058035, 0.8738781566283977, 0.8645395879852721, 0.8547833874546245, 0.8446039420107051, 0.8339961356088421, 0.822955389703642, 0.8114777055053185, 0.7995597081938626, 0.7871986933258425, 0.7743926756836963, 0.761140440832014, 0.7474415996590261, 0.7332966461936803, 0.7187070189985416, 0.7036751664452645, 0.6882046161813561, 0.67230004909276, 0.6559673780545235, 0.6392138317390095, 0.622048043714805, 0.60448014701604